In [3]:
import numpy as np

# mne imports
import mne
from mne import io
from mne.datasets import sample

# EEGNet-specific imports
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.metrics import confusion_matrix 
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt

In [4]:
K.set_image_data_format('channels_last')

In [5]:
import scipy.io

classes = scipy.io.loadmat("D:\\shared_git\\MasterThesis\\scripts\\classes.mat")['classes']
ERPs = scipy.io.loadmat("D:\\shared_git\\MasterThesis\\scripts\\ERPs.mat")['ERPs']


In [6]:
X = ERPs*1000 # format is in (trials, channels, samples)
y = classes.T[0]

In [7]:
kernels, chans, samples = 1, 22, 1250

from sklearn.model_selection import train_test_split

# Assuming ERPs is your data with the shape (trials, channels, samples)
X = ERPs*1000 # format is in (trials, channels, samples)
y = classes.T[0]

# Split the data into 50% training and 50% temp (which will later be split into test and validation)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=42)

# Split the temp data into 50% test and 50% validation, resulting in 25% of the original data each
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print(y_train)

[ 2  6  5 ...  9  2 12]


In [8]:
# convert labels to one-hot encodings.
y_train      = np_utils.to_categorical(y_train-1)
y_val   = np_utils.to_categorical(y_val-1)
y_test       = np_utils.to_categorical(y_test-1)
print(y_train)
# convert data to NHWC (trials, channels, samples, kernels) format. Data 
X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
X_val   = X_val.reshape(X_val.shape[0], chans, samples, kernels)
X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)
   
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

model = EEGNet(nb_classes = 12, Chans = chans, Samples = samples, 
               dropoutRate = 0.5, kernLength = 32, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

# count number of parameters in the model
numParams    = model.count_params()    



[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
X_train shape: (3774, 22, 1250, 1)
3774 train samples
1887 test samples


In [9]:
# the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
# the weights all to be 1
class_weights = {0:1, 1:1, 2:1, 3:1, 4:1, 5:1, 6:1, 7:1, 8:1, 9:1, 10:1, 11:1}
print(y_val)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [11]:
# set a valid path for your system to record model checkpoints
checkpointer = ModelCheckpoint(filepath='D:/shared_git/MasterThesis/scripts/Processing/NNClassification/checkpoint.keras', verbose=1,
                               save_best_only=True)

# load optimal weights
model.load_weights('D:/shared_git/MasterThesis/scripts/Processing/NNClassification/EEGNet-8-2-weights.keras')
fittedModel = model.fit(X_train, y_train, batch_size = 16, epochs = 1000, 
                        verbose = 2, validation_data=(X_val, y_val),
                        callbacks=[checkpointer], class_weight = class_weights)

# load optimal weights
model.load_weights('D:/shared_git/MasterThesis/scripts/Processing/NNClassification/checkpoint.keras')

Epoch 1/1000

Epoch 1: val_loss improved from inf to 2.47770, saving model to /tmp\checkpoint.keras
236/236 - 127s - loss: 2.4918 - accuracy: 0.0819 - val_loss: 2.4777 - val_accuracy: 0.1028 - 127s/epoch - 538ms/step
Epoch 2/1000

Epoch 2: val_loss improved from 2.47770 to 2.46274, saving model to /tmp\checkpoint.keras
236/236 - 83s - loss: 2.4614 - accuracy: 0.1203 - val_loss: 2.4627 - val_accuracy: 0.1097 - 83s/epoch - 353ms/step
Epoch 3/1000

Epoch 3: val_loss did not improve from 2.46274
236/236 - 83s - loss: 2.4474 - accuracy: 0.1290 - val_loss: 2.4647 - val_accuracy: 0.1049 - 83s/epoch - 351ms/step
Epoch 4/1000

Epoch 4: val_loss improved from 2.46274 to 2.45562, saving model to /tmp\checkpoint.keras
236/236 - 74s - loss: 2.4397 - accuracy: 0.1365 - val_loss: 2.4556 - val_accuracy: 0.1092 - 74s/epoch - 312ms/step
Epoch 5/1000

Epoch 5: val_loss did not improve from 2.45562
236/236 - 101s - loss: 2.4243 - accuracy: 0.1407 - val_loss: 2.4584 - val_accuracy: 0.1113 - 101s/epoch - 42

In [ ]:
loss = fittedModel.history['loss']
val_loss = fittedModel.history['val_loss']
epochs = range(1, len(loss) + 1)